In [ ]:
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from io import BytesIO
import config as cfg

def read_parquet_from_adls(path):
    svc = DataLakeServiceClient(
        account_url=f"https://{cfg.STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
        credential=cfg.STORAGE_ACCOUNT_KEY
    )
    fs = svc.get_file_system_client(cfg.CONTAINER_NAME)
    fc = fs.get_file_client(path)
    data = fc.download_file().readall()
    buf = BytesIO(data)
    return pd.read_parquet(buf)

df = read_parquet_from_adls("curated/trips_combined_total.parquet")
df.head()


: 